In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sqlite3
import os
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
BASE_PATH =  '/home/eric/jupyter-project/quickdraw-doodle-recognition/'
INPUT_PATH = f'{BASE_PATH}/input'

In [3]:
!mkdir -p ./data

In [4]:
def f2cat(filename):
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(f'{INPUT_PATH}/train_simplified')
    return sorted([f2cat(f) for f in files], key=str.lower)

In [5]:
categories = list_all_categories()
cat2id = {c: i for i, c in enumerate(categories)}
print(len(categories))

340


In [6]:
!rm ./data/quickdraw_dataset.db
db = sqlite3.connect('./data/quickdraw_dataset.db')
c = db.cursor()

rm: impossible de supprimer './data/quickdraw_dataset.db': Aucun fichier ou dossier de ce type


In [7]:
# Maximize performance, we suppose no system crash for the course of this notebook
_ = c.execute('PRAGMA JOURNAL_MODE=OFF')
_ = c.execute('PRAGMA SYNCHRONOUS=OFF')
_ = c.execute('PRAGMA LOCKING_MODE=EXCLUSIVE')

In [8]:
_ = c.execute('CREATE TABLE classes (y INTEGER PRIMARY KEY, label VARCHAR)')
_ = c.execute('CREATE TABLE train (key_id INTEGER PRIMARY KEY, drawing VARCHAR, recognized BOOLEAN, y INTEGER)')
_ = c.execute('CREATE TABLE test (key_id INTEGER PRIMARY KEY, drawing VARCHAR)')

In [9]:
df = pd.read_csv(f'{INPUT_PATH}/test_simplified.csv', usecols=['key_id', 'drawing'])
df.sort_values('key_id', inplace=True)
_ = c.execute('BEGIN TRANSACTION')
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    _ = c.execute('INSERT INTO test (key_id, drawing) VALUES (?,?)', (row.key_id, row.drawing))
_ = c.execute('COMMIT')

100%|██████████| 112199/112199 [00:07<00:00, 15284.33it/s]


In [10]:
for cat in tqdm(categories):
    _ = c.execute('BEGIN TRANSACTION')
    y = cat2id[cat]
    _ = c.execute('INSERT INTO classes (y, label) VALUES (?,?)', (y, cat))
    df = pd.read_csv(f'{INPUT_PATH}/train_simplified/{cat}.csv', usecols=['key_id', 'drawing', 'recognized'])
    df.sort_values('key_id', inplace=True)
    for index, row in df.iterrows():
        _ = c.execute('INSERT INTO train (key_id, drawing, recognized, y) VALUES (?,?,?,?)', (row.key_id, row.drawing, row.recognized, y))
    _ = c.execute('COMMIT')

100%|██████████| 340/340 [2:50:15<00:00, 66.05s/it]  


In [11]:
db.close()

### Sanity check

In [18]:
# open database in read only mode
db = sqlite3.connect(f'file:{BASE_PATH}/data/quickdraw_dataset.db?mode=ro', uri=True)
c = db.cursor()

In [19]:
%%time
c.execute('SELECT key_id, drawing, recognized, y FROM train LIMIT 5')
print(c.fetchone())

(4503599667216384, '[[[0, 18, 150, 216, 223, 223, 218, 210, 197, 173, 123, 75, 38, 14, 6], [27, 29, 10, 5, 18, 38, 57, 70, 81, 93, 103, 103, 87, 59, 30]], [[1, 0, 9, 24, 90, 131, 166, 191, 222, 234, 246, 249, 249, 246, 231, 209], [28, 43, 75, 97, 120, 126, 125, 120, 101, 91, 67, 52, 29, 20, 6, 0]], [[65, 68, 73], [59, 64, 65]], [[95, 121], [38, 37]], [[126, 141], [64, 63]], [[155, 164], [47, 32]], [[176, 187], [59, 52]], [[200, 203], [12, 8]], [[240, 252, 255, 254, 248, 225, 195], [16, 24, 31, 50, 65, 87, 105]], [[34, 40], [48, 52]], [[56, 76], [29, 40]], [[78, 79, 88], [73, 78, 80]], [[117, 143], [82, 84]], [[161, 173], [78, 78]], [[186, 194], [42, 28]], [[149, 151], [20, 20]], [[92, 95], [62, 62]]]', 1, 329)
CPU times: user 1.1 ms, sys: 122 µs, total: 1.22 ms
Wall time: 807 µs


In [20]:
db.close()